In [1]:
from bokeh.plotting import figure
from bokeh.io import output_notebook,show
import pandas as pd
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

#### Load AAPL stock dataset
This contains information about the Apple stock price and volume over the 3-month period from May-July 2017.

In [2]:
aapl = pd.read_csv('datasets/AAPL.csv')
aapl.head()

Date        Open        High         Low       Close    AdjClose  \
0  2017-05-01  145.100006  147.199997  144.960007  146.580002  144.297287   
1  2017-05-02  147.539993  148.089996  146.839996  147.509995  145.212799   
2  2017-05-03  145.589996  147.490005  144.270004  147.059998  144.769821   
3  2017-05-04  146.520004  147.139999  145.809998  146.529999  144.248062   
4  2017-05-05  146.759995  148.979996  146.759995  148.960007  146.640228   

     Volume  
0  33602900  
1  45352200  
2  45697000  
3  23371900  
4  27327700

#### Convert the Date values to datetime type
This will allow Bokeh to interpret this field as a Date rather than a string

In [3]:
aapl['Date'] = pd.to_datetime(aapl['Date'])

#### Scale the traded volume to millions
It is easier for us to handle smaller numbers to avoid overflow errors

In [4]:
aapl['Volume'] = aapl['Volume']/1000000

#### Examine the data
This will allow us to set the axis ranges depending on the range of the data

In [5]:
aapl.describe()

Open        High         Low       Close    AdjClose     Volume
count   63.000000   63.000000   63.000000   63.000000   63.000000  63.000000
mean   149.460477  150.484127  148.304604  149.500951  147.703029  27.620316
std      4.229400    4.075143    4.264109    4.126731    4.081040  11.259767
min    142.899994  143.500000  142.199997  142.270004  140.632507  14.258300
25%    145.510002  146.720001  144.495003  145.780006  144.102120  20.527550
50%    149.199997  150.440002  148.570007  149.559998  147.838593  24.761900
75%    153.680001  154.205001  152.650002  153.400002  151.583488  31.848300
max    156.009995  156.649994  155.050003  156.100006  154.303329  72.307300

#### Import the ColumnDataSource class
This will map names of columns to sequences or arrays which can be used in our Bokeh plots. 

In [6]:
from bokeh.models.sources import ColumnDataSource

#### A Pandas dataframe can be turned to a ColumnDataSource
This is the way to use tabular data in Bokeh plots.

In [7]:
data_source = ColumnDataSource(aapl)

#### Declare the figure
In addition to the plot dimensions, we set the following:
* We set the <b>x_axis_type</b> to 'datetime' so that Bokeh treats it as a Date
* We set the <b>y_range</b> to cover the values of the adjusted close price

In [8]:
p = figure(plot_width = 600, 
           plot_height = 300,
           
           x_axis_type = 'datetime',
           
           y_range = Range1d(135, 160)
          )

#### Draw the line representing the adjusted close vs time
Here, we specify a source for the data and reference the x and y values by the column name strings rather than pass the entire series. 

In [9]:
p.line(x = 'Date', 
       y = 'AdjClose',
       
       color = 'blue',
       
       source = data_source
      )

show(p)

#### Format the axes
We can format the X and Y axes by referencing the p.xaxis and p.yaxis objects. We mark the Y axis in blue to match the color of the line for Adj Close. 

In [10]:
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Adj Close (USD)'


p.yaxis.axis_line_width = 2

p.yaxis.axis_label_text_color = 'blue'
p.yaxis.axis_line_color = 'blue'
p.yaxis.major_label_text_color = 'blue'

#### View the updated plot

In [11]:
show(p)

## Twin Axis
We can add another axis to the plot to represent another range of values whose units are different from the values denoted on the first Y axis. For instance, we have one axis for the Adjusted Close price in USD. We can another axis to show the traded volume in millions of shares.

#### Import the LinearAxis class
This is needed for us to define a new axis

In [12]:
from bokeh.models import LinearAxis

#### Define the new Y axis
We call this one VolumeAxis and specify the range of values we would like it to cover. Given the range of shares traded is between 14M and 72M per day, we set an appropriate range.

In [13]:
p.extra_y_ranges = {'VolumeAxis' : Range1d(start=10, end=75)}

#### Draw a new line for the traded volume
Here, we need to explicitly point the line to the newly created Y axis using the <b>y_range_name</b> property. 

In [14]:
p.line(x = 'Date', 
       y = 'Volume',
       
       color = 'green',
       
       y_range_name = 'VolumeAxis',
       
       source = data_source)

GlyphRenderer(id='1338', ...)

#### Add the new Y axis to the plot
We add a new LinearAxis object to the right of the plot. This will reference the 'VolumeAxis' from the figure's extra_y_ranges

In [15]:
p.add_layout(LinearAxis(y_range_name = 'VolumeAxis'), 
             'right')

#### Format the new Y axis
p.yaxis is really a list of Y axes. Here, we reference the new axis we have created by accessing it from its index in the list. 

We set the color to match the line color for the volume.

In [16]:
p.yaxis[1].axis_label = 'Traded Volume (millions)'


p.yaxis[1].axis_line_width = 2

p.yaxis[1].axis_label_text_color = 'blue'
p.yaxis[1].axis_line_color = 'green'
p.yaxis[1].major_label_text_color = 'green'

In [17]:
show(p)